# Setup

In [ ]:
anchorfile = "~/work/experiments/bots-strassen/trace/1024.47356/1024.47356.otf2"

In [ ]:
import pandas as pd
import otter
otter.log.initialise("debug")
reader = otter.project.ReadTraceData(anchorfile).connect()
reader.count_simulation_rows()

# Simulate

In [ ]:
if reader.count_simulations() == 0:
    otter.log.initialise("info")
    otter.project.simulate_schedule(anchorfile)
    otter.log.initialise("debug")
else:
    print("Already simulated")

# Query

In [ ]:
reader.count_simulation_rows()

In [ ]:
critical_tasks = reader.get_tasks(reader.get_critical_tasks(sim_id=0))
print([task.id for task in critical_tasks])

In [ ]:
tasks = pd.DataFrame((task.asdict(flatten=True) for task in critical_tasks))
min_time = tasks["create_ts"].astype(int).min()
tasks["create_ts"] = tasks["create_ts"].astype(int) - min_time
tasks["start_ts"] = tasks["start_ts"].astype(int) - min_time
tasks["end_ts"] = tasks["end_ts"].astype(int) - min_time

In [ ]:
tasks.head(3)

In [ ]:
scheduling_states = reader.get_task_scheduling_states([task.id for task in critical_tasks])
states = pd.DataFrame((state.asdict() for state in scheduling_states))
min_time = states["start_ts"].astype(int).min()
states["start_ts"] = states["start_ts"].astype(int) - min_time
states["end_ts"] = states["end_ts"].astype(int) - min_time
states.head(8)

In [ ]:
scheduling_states_sim = reader.get_task_scheduling_states([task.id for task in critical_tasks], sim_id=0)
states_sim = pd.DataFrame((state.asdict() for state in scheduling_states_sim))
states_sim["start_ts"] = states_sim["start_ts"].astype(int)
states_sim["end_ts"] = states_sim["end_ts"].astype(int)
states_sim.head(3)

# Plot Gantt Chart

In [ ]:
import matplotlib.pyplot as plt

COLOUR_ACTIVE = (0,0,1)       # blue
COLOUR_INACTIVE = (0.8627,)*3 # grey

action_colour = {
    "CREATE": COLOUR_INACTIVE,
    "START": COLOUR_ACTIVE,
    "SUSPEND": COLOUR_INACTIVE,
    "RESUME": COLOUR_ACTIVE,
}

## Plot scheduling states of native & simulated schedules

In [ ]:
def plot_task_scheduling_states(tasks, states):

    fig, ax = plt.subplots()
    
    # plot tasks in the order they start:
    start_ts_df = tasks.sort_values("start_ts")[["id"]]
    
    task_labels = []
    yticks = []
    for index, task_id in enumerate(start_ts_df["id"]):
        task_labels.append(task_id)
        yticks.append(index+1.25)
        task_rows = states[states["task"]==task_id]
        xranges = [_ for _ in zip(task_rows["start_ts"].to_list(), task_rows["duration"].to_list(), strict=True)]
        colours = [action_colour[action] for action in task_rows["action_start"]]
        assert len(task_rows) == len(colours)
        colours = tuple(action_colour[action] for action in task_rows['action_start'])
        ax.broken_barh(xranges=xranges, yrange=(index+1, 0.5), facecolors=colours)
    
    ax.set_yticks(yticks)
    ax.set_yticklabels(task_labels)
    
    plt.gca().invert_yaxis()

In [ ]:
plot_task_scheduling_states(tasks, states)

In [ ]:
plot_task_scheduling_states(tasks, states_sim)